## **Meeting Scheduler**

In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict
from datetime import datetime, timedelta
import pytz

class MeetingTimeOptimizer:
    def __init__(self, data: pd.DataFrame):
        """
        Initialize the Meeting Time Optimizer
        
        Args:
            data (pd.DataFrame): DataFrame containing employee information
        """
        self.df = data
        self.timezones = {}
        self._parse_timezones()
    
    def _parse_timezones(self):
        """
        Parse and standardize timezone information
        """
        for _, row in self.df.iterrows():
            try:
                # Convert timezone strings to pytz objects
                tz = pytz.timezone(row['timezone'])
                self.timezones[row['employee_id']] = {
                    'timezone': tz,
                    'utc_offset': self._get_utc_offset(tz),
                    'location': row['location'],
                    'department': row['department'],
                    'seniority_level': row['seniority_level']
                }
            except Exception as e:
                print(f"Error processing timezone for {row['employee_id']}: {e}")
    
    def _get_utc_offset(self, tz) -> float:
        """
        Calculate UTC offset for a given timezone
        
        Args:
            tz (pytz.timezone): Timezone object
        
        Returns:
            float: UTC offset in hours
        """
        # Get current UTC offset
        return datetime.now(tz).utcoffset().total_seconds() / 3600
    
    def recommend_meeting_time(self, employee_ids: List[int]) -> Dict:
        """
        Recommend optimal meeting time considering multiple constraints
        
        Args:
            employee_ids (List[int]): List of employee IDs to include in meeting
        
        Returns:
            Dict: Recommended meeting details
        """
        # Validate employee IDs
        valid_employees = [emp_id for emp_id in employee_ids 
                           if emp_id in self.timezones]
        
        if not valid_employees:
            raise ValueError("No valid employees found")
        
        # Extract timezone information for selected employees
        selected_timezones = {
            emp_id: self.timezones[emp_id] for emp_id in valid_employees
        }
        
        # Calculate offset statistics
        offsets = [info['utc_offset'] for info in selected_timezones.values()]
        
        # Optimal time calculation strategies
        def calculate_optimal_time():
            # Strategy 1: Median offset
            median_offset = np.median(offsets)
            
            # Strategy 2: Weighted offset considering seniority and department
            def weight_offset(emp_id):
                emp_info = selected_timezones[emp_id]
                seniority_weights = {
                    'junior': 0.7,
                    'mid': 1.0,
                    'senior': 1.3
                }
                return (
                    emp_info['utc_offset'] * 
                    seniority_weights.get(emp_info['seniority_level'], 1.0)
                )
            
            weighted_offsets = [weight_offset(emp_id) for emp_id in valid_employees]
            weighted_median = np.median(weighted_offsets)
            
            return weighted_median
        
        # Determine optimal meeting time
        optimal_offset = calculate_optimal_time()
        
        # Validate time is within reasonable working hours
        def validate_working_hours(offset):
            # Assume standard working hours are 9 AM - 5 PM
            return -6 <= offset <= 12
        
        if not validate_working_hours(optimal_offset):
            # Fallback to a more moderate time
            optimal_offset = max(-6, min(optimal_offset, 12))
        
        # Prepare recommendation details
        recommendation = {
            'recommended_utc_offset': f'UTC{int(optimal_offset):+d}',
            'participants': [],
            'offset_calculation': {
                'median_offset': np.median(offsets),
                'weighted_median_offset': optimal_offset
            }
        }
        
        # Add participant details
        for emp_id in valid_employees:
            emp_info = selected_timezones[emp_id]
            recommendation['participants'].append({
                'employee_id': emp_id,
                'timezone': str(emp_info['timezone']),
                'location': emp_info['location'],
                'local_time_at_meeting': self._convert_to_local_time(
                    optimal_offset, 
                    emp_info['timezone']
                )
            })
        
        return recommendation
    
    def _convert_to_local_time(self, target_offset, local_tz):
        """
        Convert target UTC time to local timezone
        
        Args:
            target_offset (float): Target UTC offset
            local_tz (pytz.timezone): Local timezone
        
        Returns:
            str: Formatted local time
        """
        # Create a UTC time with the target offset
        utc_time = datetime.utcnow().replace(
            hour=10, minute=0, second=0, microsecond=0  # Assume 10 AM UTC
        )
        utc_time += timedelta(hours=target_offset)
        
        # Convert to local timezone
        local_time = utc_time.astimezone(local_tz)
        
        return local_time.strftime('%I:%M %p')

In [2]:
def scheduler(emp_list:list, dataset):
    # Load data (replace with your actual data loading method)
    data = pd.read_excel(dataset)  # Or use the provided dataframe
    
    # Create optimizer
    optimizer = MeetingTimeOptimizer(data)
    
    # Example usage
    try:
        # Recommend meeting time for specific employees
        recommendation = optimizer.recommend_meeting_time(emp_list)
        
        # Print recommendation details
        print("========== Meeting Time Recommendation ==========")
        print(f"Recommended UTC Offset: {recommendation['recommended_utc_offset']} \n")
        print("========== Participant Local Times ==========")
        for participant in recommendation['participants']:
            print(f"Employee {participant['employee_id']} ({participant['timezone']}):")
            print(f"  Location: {participant['location']}")
            print(f"  Local Meeting Time: {participant['local_time_at_meeting']}\n")
        
        print("Offset Calculation Details:")
        print(recommendation['offset_calculation'])
    
    except Exception as e:
        print(f"Error in meeting time recommendation: {e}")

In [3]:
# Example Implementation
scheduler([21, 28], 'globalsync.xlsx')

========== Meeting Time Recommendation ==========
Recommended UTC Offset: UTC+1 

========== Participant Local Times ==========
Employee 21 (Europe/Berlin):
  Location: Hamburg, Germany
  Local Meeting Time: 06:30 AM

Employee 28 (Europe/Prague):
  Location: Prague, Czech Republic
  Local Meeting Time: 06:30 AM

Offset Calculation Details:
{'median_offset': 1.0, 'weighted_median_offset': 1.0}
